## Импорты

In [4]:
# !pip3 install matplotlib tqdm numpy torch

In [5]:
# %matplotlib inline
# import matplotlib.pyplot as plt

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader

import os
from tqdm import tqdm_notebook, trange, tqdm
import pickle
import numpy as np
from IPython.display import clear_output
import jedi
import tokenize
from io import BytesIO
import timeit

In [2]:
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [3]:
# device

device(type='cpu')

# Playing with jedi module

In [3]:
code = '''from __future__ import print_function
from socket import timeout
import os
import sys
import codecs
import re
import markdown
try:
    from urllib2 import urlopen
    from urllib2 import HTTPError
    from urllib2 import URLError
except ImportError:
    from urllib.request import urlopen
    from urllib.error import HTTPError
    from urllib.error import URLError


def function874(arg89):
    var4252 = False
    try:
        var4262 = urlopen(arg89, timeout=2)
        var4252 = var4262.code == 200
    except HTTPError as var3998:
        print(var3998, file=sys.stderr)
    except URLError as var3998:
        print(var3998, file=sys.stderr)
    except timeout as var3998:
        print(var3998, file=sys.stderr)
    except Exception as var3998:
        print(var3998, file=sys.stderr)
    return v'''

script = jedi.Script(code)#source, 3, len('datetime.da'), 'example.py')
script.completions()


[<Completion: var3998>,
 <Completion: var4252>,
 <Completion: var4262>,
 <Completion: vars>]

In [3]:
code = '''
def foo(i):
    f'''

script = jedi.Script(code)#source, 3, len('datetime.da'), 'example.py')
script.completions(), dir(script)


([<Completion: filter>,
  <Completion: float>,
  <Completion: foo>,
  <Completion: for>,
  <Completion: format>,
  <Completion: from>,
  <Completion: frozenset>],
 ['__class__',
  '__delattr__',
  '__dict__',
  '__dir__',
  '__doc__',
  '__eq__',
  '__format__',
  '__ge__',
  '__getattribute__',
  '__gt__',
  '__hash__',
  '__init__',
  '__init_subclass__',
  '__le__',
  '__lt__',
  '__module__',
  '__ne__',
  '__new__',
  '__reduce__',
  '__reduce_ex__',
  '__repr__',
  '__setattr__',
  '__sizeof__',
  '__str__',
  '__subclasshook__',
  '__weakref__',
  '_analysis',
  '_code',
  '_code_lines',
  '_evaluator',
  '_get_module',
  '_grammar',
  '_module_node',
  '_orig_path',
  '_path',
  '_pos',
  '_project',
  'call_signatures',
  'completions',
  'goto_assignments',
  'goto_definitions',
  'path',
  'usages'])


* суммирую все эмбеддинги, которые встретил до текущего ключевого слова, конкатенирую к эмбеддингу текущего слова
    - итерируюсь по файлам из train_files.txt
    - иду с помощью tokenize + isidentifier(), выбираю из кода identifiers, которые есть в mapping.map
        - может ли регулярка возвращать индекс начала распознанного слова?
        - сохраняю в массив `[[identifier, index], [...], ...]`
    - предсохраняю номера токенов, которые войдут в сумму
    - подаю в jedi предыдущий код + первую букву identifier'а (индекс от регулярки + 1)
    - итерируюсь по предсказаниям и добавляю объекты в train такого вида:
    ```
    [
        arr_of_prev_tokens,
        predicted_token (число из mapping.map),
        real_token (число из mapping.map)
    ]```
    - дамплю в пикл `{file_name: train}`
* подаю на вход полносвязному слою
* беру сигмоиду
* подаю в кросс-энтропию

In [2]:
times_dict = {
    'code_read': [],
    'tokenize': [],
    'before_jedi': [],
    'jedi': [],
    'after_jedi': [],
    'everything_in_for_in_g_body': [],
    'save_preprocessed': [],
}


def prepare_samples(path, samples_path):
    def get_filenames(path):
        with open(path) as f:
            filenames = f.read().split('\n')
        return filenames
    
    def load_mapping(path):
        with open(path, 'rb') as f:
#         print(base_path + '/mapping.map')
            word_to_id = pickle.load(f)
        return word_to_id

    def predict_tokens(code):
        try:
            script = jedi.Script(code)
            return script.completions()
        except KeyError:
            print(code)
            raise
    
    def load_preprocessed(path):
        try:
            with open(path, 'rb') as f:
                samples = pickle.load(f)
        except FileNotFoundError:
            samples = {}
        return samples
    
    def save_preprocessed(samples_dict_path, samples_dict):
        with open(samples_dict_path, 'wb') as f:
            samples = pickle.dump(samples_dict, f)
    
    filenames = get_filenames(path + samples_path)
    word_to_id = load_mapping('./mapping.map')
    
    samples_dict_path = './ranking_samples_' + samples_path[:3]
    samples_dict = load_preprocessed(samples_dict_path)

#     start_time = timeit.default_timer()
#     # code you want to evaluate
#     elapsed = timeit.default_timer() - start_time

    for i, filename in enumerate(tqdm(filenames)):
        
        if filename in samples_dict:
            continue

        clear_output()
        times_dict['code_read'].append(0.)
        times_dict['tokenize'].append(0.)
        times_dict['before_jedi'].append(0.)
        times_dict['jedi'].append(0.)
        times_dict['after_jedi'].append(0.)
        times_dict['everything_in_for_in_g_body'].append(0.)
        times_dict['save_preprocessed'].append(0.)

        start_time = timeit.default_timer()
        with open(path + filename) as f:
            code = f.read()
        times_dict['code_read'][-1] += timeit.default_timer() - start_time
        print(times_dict['code_read'])
        
        start_time = timeit.default_timer()
        samples = []
        prev_tokens = []
        g = tokenize.tokenize(BytesIO(code.encode("utf-8")).readline)
        code_lines = code.split('\n')
        times_dict['tokenize'][-1] += timeit.default_timer() - start_time
        print(times_dict['tokenize'])
        
        start_time = timeit.default_timer()
        for toktype, tokval, start, _, _ in g:
            if toktype == tokenize.NAME and tokval in word_to_id:
                inner_start_time = timeit.default_timer()
                row, col = start
                real_token = word_to_id[tokval]
                prev_code = '\n'.join(code_lines[:row-1]) + '\n' + code_lines[row-1][:col+1]
                times_dict['before_jedi'][-1] += timeit.default_timer() - inner_start_time
                
                inner_start_time = timeit.default_timer()
                preds = predict_tokens(prev_code)
                times_dict['jedi'][-1] += timeit.default_timer() - inner_start_time

                inner_start_time = timeit.default_timer()
                for pred in preds:
                    completion = pred.full_name
                    if completion in word_to_id:
                        completion_id = word_to_id[completion]
                        samples.append([
                            prev_tokens[:],
                            completion_id,
                            real_token
                        ])
                prev_tokens.append(real_token)
                times_dict['after_jedi'][-1] += timeit.default_timer() - inner_start_time
#                     break
        times_dict['everything_in_for_in_g_body'][-1] += timeit.default_timer() - start_time
        print(times_dict['everything_in_for_in_g_body'])
        samples_dict[filename] = samples
        
        start_time = timeit.default_timer()
        save_preprocessed(samples_dict_path, samples_dict)
        times_dict['save_preprocessed'][-1] += timeit.default_timer() - start_time
#         break
    save_preprocessed(samples_dict_path, samples_dict)
    return samples_dict

path = './pycodesuggest_py_repos_normalised/'
samples_path = 'train_files.txt'
samples = prepare_samples(path, samples_path)
        

[0.0021002540000001346, 0.0040657039999985045, 0.004112184999996771, 0.0010922739999728037, 0.001201352999999017, 0.000864787999944383, 0.0006555749999961336, 0.0010110430000054293, 0.0011278509999783637, 0.08087101799992524, 0.0006236240001271653, 0.0018655849999049678, 0.013776576999816825, 0.00041471600002296327, 0.001098770999988119, 0.0014301409999006864, 0.0008996050000860123, 0.0014114690000042174, 0.0009126420000029611, 0.0009328109999842127, 0.0006261469998207758, 0.00133860599999025, 0.0008457900000848895, 0.0009099769999920682, 0.004566714000020511, 0.0012994339999750082, 0.00109186800000316, 0.0011106520000794262, 0.004105575000039607, 0.0004915319998417544, 0.0023259320000761363, 0.0037616270001308294, 0.0027956850001373823, 0.004676811000081216, 0.0008892020000530465]
[9.641599999987704e-05, 0.0001996440000198163, 0.00013618899998846246, 0.00018906600001855622, 0.00011942399999043118, 6.030499997677907e-05, 9.052400002929062e-05, 0.00015657400001600763, 0.0007245489999831

InvalidPythonEnvironment: Could not get version information for '/usr/local/opt/python/bin/python3.7': BlockingIOError(35, 'Resource temporarily unavailable')

In [3]:
times_dict

{'code_read': [0.0021002540000001346,
  0.0040657039999985045,
  0.004112184999996771,
  0.0010922739999728037,
  0.001201352999999017,
  0.000864787999944383,
  0.0006555749999961336,
  0.0010110430000054293,
  0.0011278509999783637,
  0.08087101799992524,
  0.0006236240001271653,
  0.0018655849999049678,
  0.013776576999816825,
  0.00041471600002296327,
  0.001098770999988119,
  0.0014301409999006864,
  0.0008996050000860123,
  0.0014114690000042174,
  0.0009126420000029611,
  0.0009328109999842127,
  0.0006261469998207758,
  0.00133860599999025,
  0.0008457900000848895,
  0.0009099769999920682,
  0.004566714000020511,
  0.0012994339999750082,
  0.00109186800000316,
  0.0011106520000794262,
  0.004105575000039607,
  0.0004915319998417544,
  0.0023259320000761363,
  0.0037616270001308294,
  0.0027956850001373823,
  0.004676811000081216,
  0.0008892020000530465],
 'tokenize': [9.641599999987704e-05,
  0.0001996440000198163,
  0.00013618899998846246,
  0.00018906600001855622,
  0.000119

In [20]:
for i in tqdm(range(1000)):
#     timeit.sleep(1)
    clear_output()
    print(i)
    

100%|██████████| 1000/1000 [00:02<00:00, 407.96it/s]

999


In [4]:
with open('./ranking_samples_tra', 'rb') as f:
    samples = pickle.load(f)

In [6]:
count = 0

for v in samples.values():
    count += len(v)
print(count)
print(len(samples.values()))

76139
87


In [4]:
with open('./ranking_samples_tra', 'rb') as f:
    samples = pickle.load(f)

In [9]:
count = 0

for v in samples.values():
    count += len(v)
print(count)
print(len(samples.values()))

18534
32


In [15]:
import tokenize
from io import BytesIO
from keyword import iskeyword

def load_code(path):
    pass

def process_code(code):
#     s = code #
    s = "def twoπ(a,b):"
    g = tokenize.tokenize(BytesIO(s.encode("utf-8")).readline)
    new_s = ''
    for toktype, tokval, st, end, _ in g:
        if toktype == tokenize.NAME:
            if iskeyword(tokval):
                print("KEYWORD", tokval, tokval.isidentifier(), st)
            else:
                print('NAME', tokval, tokval.isidentifier(), st)
        else:
            print(toktype, tokval)
        new_s += f'{tokval} '
    print(new_s)

process_code(None)

57 utf-8
KEYWORD def True (1, 0)
NAME twoπ True (1, 4)
53 (
NAME a True (1, 9)
53 ,
NAME b True (1, 11)
53 )
53 :
4 
0 
utf-8 def twoπ ( a , b ) :   


In [35]:
print(new_s[8:].strip())

def foo ( i ) : 
      f


In [36]:
script = jedi.Script(new_s[8:].strip())
script.completions()

[<Completion: filter>,
 <Completion: float>,
 <Completion: foo>,
 <Completion: for>,
 <Completion: format>,
 <Completion: from>,
 <Completion: frozenset>]